## Step 1: Download Data from TFL 

In [7]:
import os
import requests
from tqdm import tqdm
from datetime import datetime, timedelta

BASE_URL = "https://cycling.data.tfl.gov.uk/usage-stats/"
TARGET_DIR = "tfl_cycle_data"
START_DATE = datetime(2020, 12, 30)
END_DATE = datetime(2023, 6, 19)  # Just before 375
START_INDEX = 246

os.makedirs(TARGET_DIR, exist_ok=True)

def format_date(d):
    return d.strftime("%d%b%Y")

current_date = START_DATE
file_index = START_INDEX
download_links = []

# === PHASE 1: Weekly files (246 to 374, with exceptions)
while current_date < END_DATE:
    # Special case for 334 (5-day file)
    if file_index == 334:
        filename = f"{file_index}JourneyDataExtract07Sep2022-11Sep2022.csv"
        download_links.append((BASE_URL + filename, filename))
        current_date = datetime(2022, 9, 12)
        file_index += 1
        continue

    # Normal 7-day range
    start_str = format_date(current_date)
    end_str = format_date(current_date + timedelta(days=6))
    filename = f"{file_index}JourneyDataExtract{start_str}-{end_str}.csv"
    download_links.append((BASE_URL + filename, filename))
    current_date += timedelta(days=7)
    file_index += 1

# Add special case for 375 (12-day file)
download_links.append((
    BASE_URL + "375JourneyDataExtract19Jun2023-30Jun2023.csv",
    "375JourneyDataExtract19Jun2023-30Jun2023.csv"
))

# === PHASE 2: Manually define filenames from 376 onwards 
manual_files = [
    "376JourneyDataExtract01Jul2023-14Jul2023.csv",
    "377JourneyDataExtract15Jul2023-31Jul2023.csv",
    "378JourneyDataExtract01Aug2023-14Aug2023.csv",
    "378JourneyDataExtract15Aug2023-31Aug2023.csv",
    "379JourneyDataExtract01Sep2023-14Sep2023.csv",
    "380JourneyDataExtract15Sep2023-30Sep2023.csv",
    "381JourneyDataExtract01Oct2023-14Oct2023.csv",
    "382JourneyDataExtract15Oct2023-31Oct2023.csv",
    "383JourneyDataExtract01Nov2023-14Nov2023.csv",
    "384JourneyDataExtract15Nov2023-30Nov2023.csv",
    "385JourneyDataExtract01Dec2023-14Dec2023.csv",
    "386JourneyDataExtract15Dec2023-31Dec2023.csv",
    "387JourneyDataExtract01Jan2024-14Jan2024.csv",
    "388JourneyDataExtract15Jan2024-31Jan2024.csv"
]

for file in manual_files:
    download_links.append((BASE_URL + file, file))

# === DOWNLOAD BLOCK
print(f"Attempting to download {len(download_links)} files...")

for url, filename in tqdm(download_links):
    path = os.path.join(TARGET_DIR, filename)
    if os.path.exists(path):
        continue
    try:
        with requests.get(url, stream=True) as r:
            r.raise_for_status()
            with open(path, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
    except Exception as e:
        print(f"❌ Failed to download {filename}: {e}")

print("✅ All available files downloaded to:", TARGET_DIR)


Attempting to download 144 files...


100%|██████████████████████████████████████████████████████████████████████████████████████████| 144/144 [04:36<00:00,  1.92s/it]

✅ All available files downloaded to: tfl_cycle_data


In [11]:
import os
import pandas as pd

# Path to your downloaded CSVs
data_dir = "tfl_cycle_data"

# Get all .csv files
csv_files = [f for f in os.listdir(data_dir) if f.endswith(".csv")]

# Combine them into one DataFrame
combined_df = pd.DataFrame()
for file in csv_files:
    path = os.path.join(data_dir, file)
    try:
        df = pd.read_csv(path , low_memory=False)
        df["source_file"] = file  # Track file of origin
        combined_df = pd.concat([combined_df, df], ignore_index=True)
    except Exception as e:
        print(f"Error reading {file}: {e}")

# Save combined file (optional)
combined_df.to_csv("combined_tfl_cycle_data.csv", index=False)
print("✅ Combined CSV saved as combined_tfl_cycle_data.csv")


✅ Combined CSV saved as combined_tfl_cycle_data.csv


In [ ]:
import pandas as pd 

combined_df = pd.read_csv("combined_tfl_cycle_data.csv", low_memory=False)

combined_df.head()

In [19]:
combined_df = pd.read_csv("combined_tfl_cycle_data.csv", index=False,low_memory=False)

# Define groups of semantically duplicate columns
column_groups = {
    "rental_id": ["Rental Id"],
    "bike_id": ["Bike Id", "Bike number"],
    "start_time": ["Start Date", "Start date"],
    "end_time": ["End Date", "End date"],
    "start_station_id": ["StartStation Id", "Start station number"],
    "end_station_id": ["EndStation Id", "End station number"],
    "start_station_name": ["StartStation Name", "Start station"],
    "end_station_name": ["EndStation Name", "End station"],
    "duration_sec": ["Duration", "Total duration"],
    "duration_ms": ["Total duration (ms)"],
    "bike_model": ["Bike model"],
    "bike_internal_id": ["Number"],
    "source_file": ["source_file"]
}

for unified_col, candidates in column_groups.items():
    print(f"\n📌 Checking types for unified column: '{unified_col}'")
    for col in candidates:
        if col in combined_df.columns:
            type_counts = combined_df[col].map(lambda x: type(x).__name__).value_counts()
            print(f"  - {col}: {dict(type_counts)}")
        else:
            print(f"  - {col}: ❌ Not present in DataFrame")

            
cleaned_df = pd.DataFrame()
for unified_col, candidates in column_groups.items():
    for col in candidates:
        if col in combined_df.columns:
            # Use the first available non-null value
            if unified_col not in cleaned_df:
                cleaned_df[unified_col] = combined_df[col]
            else:
                cleaned_df[unified_col] = cleaned_df[unified_col].combine_first(combined_df[col])



📌 Checking types for unified column: 'rental_id'
  - Rental Id: ❌ Not present in DataFrame

📌 Checking types for unified column: 'bike_id'
  - Bike Id: ❌ Not present in DataFrame
  - Bike number: ❌ Not present in DataFrame

📌 Checking types for unified column: 'start_time'
  - Start Date: ❌ Not present in DataFrame
  - Start date: ❌ Not present in DataFrame

📌 Checking types for unified column: 'end_time'
  - End Date: ❌ Not present in DataFrame
  - End date: ❌ Not present in DataFrame

📌 Checking types for unified column: 'start_station_id'
  - StartStation Id: ❌ Not present in DataFrame
  - Start station number: ❌ Not present in DataFrame

📌 Checking types for unified column: 'end_station_id'
  - EndStation Id: ❌ Not present in DataFrame
  - End station number: ❌ Not present in DataFrame

📌 Checking types for unified column: 'start_station_name'
  - StartStation Name: ❌ Not present in DataFrame
  - Start station: ❌ Not present in DataFrame

📌 Checking types for unified column: 'end_

In [ ]:
import pandas as pd 

combined_df = pd.read_csv("combined_tfl_cycle_data.csv", low_memory=False)

# List of columns you want to inspect
columns_to_check = combined_df.columns

# Inspect each column
for col in columns_to_check:
    print(f"\n📌 Column: {col}")
    
    if col not in combined_df.columns:
        print("❌ Not found in DataFrame")
        continue
    
    # Data type summary
    try:
        type_counts = combined_df[col].map(lambda x: type(x).__name__).value_counts()
        print(f"🔎 Type Distribution: {dict(type_counts)}")
    except Exception as e:
        print(f"⚠️ Error getting type counts: {e}")

    # Sample values
    try:
        sample_vals = combined_df[col].dropna().unique()[:5]
        print("🧪 Sample Values:", sample_vals)
    except Exception as e:
        print(f"⚠️ Error getting sample values: {e}")


In [12]:
combined_df.columns

Index(['Rental Id', 'Duration', 'Bike Id', 'End Date', 'EndStation Id',
       'EndStation Name', 'Start Date', 'StartStation Id', 'StartStation Name',
       'source_file', 'Number', 'Start date', 'Start station number',
       'Start station', 'End date', 'End station number', 'End station',
       'Bike number', 'Bike model', 'Total duration', 'Total duration (ms)'],
      dtype='object')

In [13]:
# Mapping from raw to unified column names
column_mapping = {
    'Rental Id': 'rental_id',
    'Bike Id': 'bike_id',
    'Bike number': 'bike_id',
    'Bike model': 'bike_model',

    'Start Date': 'start_time',
    'Start date': 'start_time',
    'End Date': 'end_time',
    'End date': 'end_time',

    'StartStation Id': 'start_station_id',
    'Start station number': 'start_station_id',
    'EndStation Id': 'end_station_id',
    'End station number': 'end_station_id',

    'StartStation Name': 'start_station_name',
    'Start station': 'start_station_name',
    'EndStation Name': 'end_station_name',
    'End station': 'end_station_name',

    'Duration': 'duration_sec',
    'Total duration': 'duration_sec',
    'Total duration (ms)': 'duration_ms',
    
    'Number': 'bike_internal_id',  # if distinct
    'source_file': 'source_file'
}

# Apply mapping if column exists
combined_df = combined_df.rename(columns={col: new for col, new in column_mapping.items() if col in combined_df.columns})


In [16]:
type_report = {}

for col in combined_df.columns:
    try:
        # Only proceed if it's a proper Series (1D)
        if combined_df[col].ndim == 1:
            type_counts = combined_df[col].map(lambda x: type(x).__name__).value_counts()
            type_report[col] = type_counts
        else:
            print(f"⚠️ Skipped non-1D column: {col}")
    except Exception as e:
        print(f"❌ Error processing column {col}: {e}")

# Combine into readable DataFrame
type_summary_df = pd.DataFrame(type_report).T.fillna(0).astype(int)
type_summary_df.index.name = "column"
type_summary_df.columns.name = "python_type"
type_summary_df = type_summary_df.sort_index()

print("✅ Type consistency analysis complete.")
print(type_summary_df)

⚠️ Skipped non-1D column: duration_sec
⚠️ Skipped non-1D column: bike_id
⚠️ Skipped non-1D column: end_time
⚠️ Skipped non-1D column: end_station_id
⚠️ Skipped non-1D column: end_station_name
⚠️ Skipped non-1D column: start_time
⚠️ Skipped non-1D column: start_station_id
⚠️ Skipped non-1D column: start_station_name
⚠️ Skipped non-1D column: start_time
⚠️ Skipped non-1D column: start_station_id
⚠️ Skipped non-1D column: start_station_name
⚠️ Skipped non-1D column: end_time
⚠️ Skipped non-1D column: end_station_id
⚠️ Skipped non-1D column: end_station_name
⚠️ Skipped non-1D column: bike_id
⚠️ Skipped non-1D column: duration_sec
✅ Type consistency analysis complete.
python_type          float       str
column                              
bike_internal_id  31344479         0
bike_model        19727277  11617202
duration_ms       31344479         0
rental_id         31344479         0
source_file              0  31344479


In [18]:
from collections import Counter
print(Counter(combined_df.columns))

Counter({'duration_sec': 2, 'bike_id': 2, 'end_time': 2, 'end_station_id': 2, 'end_station_name': 2, 'start_time': 2, 'start_station_id': 2, 'start_station_name': 2, 'rental_id': 1, 'source_file': 1, 'bike_internal_id': 1, 'bike_model': 1, 'duration_ms': 1})
